In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
cwd = os.getcwd()

api_path_сifar = os.path.join(cwd, '..', 'tools', 'cifar10_utils')
api_path_sampling = os.path.join(cwd, '..', 'tools', 'sampling_utils')
api_path_gan_metrics = os.path.join(cwd, '..', 'tools', 'gan_metrics')
models_cifar_scratch_path = os.path.join(cwd, '..', 'models', 'models_cifar10')

sys.path.append(api_path_сifar)
sys.path.append(api_path_sampling)
sys.path.append(api_path_gan_metrics)

In [3]:
import numpy as np
import time
import random
import glob
import importlib
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from distributions import IndependentNormal
import torchvision.transforms as transforms

from functools import partial

from dataloader import get_loader

from dcgan import (Discriminator_cifar10, 
                   Generator_cifar10)

from params_cifar10 import args
from general_utils import to_var, DotDict, to_np, Discriminator_logits

from metrics_utils import calculate_images_statistics, z_transform

from mh_sampling import mh_sampling, mh_sampling_from_scratch
from ebm_sampling import (langevin_sampling, 
                          aggregate_sampling_output,
                          load_data_from_batches,
                          mala_sampling,
                          tempered_transitions_sampling,
                          gan_energy)

from sir_ais_sampling import (sir_independent_sampling,
                              sir_correlated_sampling)

from metrics import inception_score
from dataloader import GenDataset, LatentFixDataset

## Download model

In [4]:
args.device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = args.device

G = Generator_cifar10(ngpu=1)
D = Discriminator_cifar10(ngpu=1)

D.load_state_dict(torch.load(os.path.join(models_cifar_scratch_path, 'netD_epoch_199.pth')))
G.load_state_dict(torch.load(os.path.join(models_cifar_scratch_path, 'netG_epoch_199.pth')))

D_logits = Discriminator_logits(D, ngpu=1)

if torch.cuda.is_available():
    D = D.to(device).eval()
    G = G.to(device).eval()
    D_logits = D_logits.to(device).eval()
    
G.z_dim = 100
G.device = device
z_dim = 100

In [5]:
loc = torch.zeros(z_dim).to(device)
scale = torch.ones(z_dim).to(device)

proposal_args = DotDict()
proposal_args.device = device
proposal_args.loc = loc
proposal_args.scale = scale
proposal = IndependentNormal(proposal_args)

log_prob = True
normalize_to_0_1 = True

def z_transform(z):
    return z.unsqueeze(-1).unsqueeze(-1)

target_gan = partial(gan_energy,
                     generator = G, 
                     discriminator = D_logits, 
                     proposal = proposal,
                     normalize_to_0_1 = normalize_to_0_1,
                     log_prob = log_prob,
                     z_transform = z_transform)

## ULA

In [5]:
method_name = 'ula'
batch_size = 1000
n = 50000
n_steps = 1000
grad_step = 0.01
eps_scale = 0.1
path_to_np_files = '/home/daniil/gans-mcmc/saved_numpy_arrays'
history_file_name_ula_small = f'{method_name}_eps_{grad_step}_noise_scale_{eps_scale}_nsteps_{n_steps}_small.npy'
path_to_read_ula_latent_vectors_small = os.path.join(path_to_np_files, history_file_name_ula_small)
z_agg_step = np.load(path_to_read_ula_latent_vectors_small)

In [6]:
z_agg_step.shape

(21, 50000, 100)

In [7]:
batch_size = 100
random_seed = 42
path_to_save_np = '/home/daniil/gans-mcmc/saved_numpy_arrays'
path_to_save = path_to_save_np
method_name = 'ula_dcgan_recalculate'
every_step = 50
dataset = "cifar10"
calculate_is = False

ula_statistics = calculate_images_statistics(z_agg_step = z_agg_step, 
                                             G = G,
                                             device = device, 
                                             batch_size = batch_size,
                                             path_to_save = path_to_save,
                                             path_to_save_np = path_to_save_np,
                                             method_name = method_name,
                                             random_seed = random_seed,
                                             every_step = every_step,
                                             dataset = dataset,
                                             calculate_is = calculate_is)

Files already downloaded and verified
Files already downloaded and verified
------------------------------------
step = 0
sample size = (50000, 100)
sample size after deleteting nans = (50000, 100)
start to calculate FID score for test cifar10...


100it [00:01, 55.27it/s]


shape of generated images = (10000, 3, 32, 32)
shape of real images = (10000, 3, 32, 32)


  1%|          | 2/200 [00:00<00:16, 12.03it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_recalculate_pretrained_fake_test_step_0.npy


...

shape of generated images = (50000, 3, 32, 32)
shape of real images = (50000, 3, 32, 32)


  0%|          | 2/1000 [00:00<01:27, 11.40it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_recalculate_pretrained_fake_train_step_1000.npy


fid: 100%|██████████| 10/10 [01:14<00:00,  7.41s/it, mean=41.9]

FID score for train cifar10 with ula_dcgan_recalculate: mean 41.87512163122073, std 0.2449252218579044
time for FID calculation on train = 257.466s


In [10]:
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

batch_size = 1000
n = 50000
n_steps = 1000
grad_step = 0.01
eps_scale = 0.1

method_name = 'ula_dcgan_cifar_recalc'
path_to_save = '/home/daniil/gans-mcmc/saved_numpy_arrays'
file_name = f'{method_name}_eps_{grad_step}_noise_scale_{eps_scale}_nsteps_{n_steps}.npy'
every_step = 50
continue_z = None

z_last_np, zs = langevin_sampling(target_gan, proposal, batch_size, n,
                                  path_to_save, file_name, every_step,
                                  continue_z,
                                  n_steps, grad_step, eps_scale)

  2%|▏         | 1/50 [04:15<3:29:03, 255.99s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_eps_0.01_noise_scale_0.1_nsteps_1000.npy_batch_num_0.npy was saved, file shape = (21, 1000, 100)


  4%|▍         | 2/50 [08:41<3:27:02, 258.81s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_eps_0.01_noise_scale_0.1_nsteps_1000.npy_batch_num_1000.npy was saved, file shape = (21, 1000, 100)


  6%|▌         | 3/50 [13:06<3:24:06, 260.57s/it]

...

file /home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_eps_0.01_noise_scale_0.1_nsteps_1000.npy_batch_num_47000.npy was saved, file shape = (21, 1000, 100)


 98%|█████████▊| 49/50 [3:35:38<04:23, 263.86s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_eps_0.01_noise_scale_0.1_nsteps_1000.npy_batch_num_48000.npy was saved, file shape = (21, 1000, 100)


100%|██████████| 50/50 [3:40:02<00:00, 264.06s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_eps_0.01_noise_scale_0.1_nsteps_1000.npy_batch_num_49000.npy was saved, file shape = (21, 1000, 100)


In [5]:
path_to_save = '/home/daniil/gans-mcmc/saved_numpy_arrays'
batch_size = 1000
n = 50000
method_name = 'ula_dcgan_cifar_recalc'
grad_step = 0.01
eps_scale = 0.1
n_steps = 1000
file_name = f'{method_name}_eps_{grad_step}_noise_scale_{eps_scale}_nsteps_{n_steps}.npy'
load_np = load_data_from_batches(n, batch_size,
                                 path_to_save, file_name)

100%|██████████| 50/50 [00:00<00:00, 283.30it/s]


In [6]:
load_np.shape

(21, 50000, 100)

In [7]:
batch_size = 100
random_seed = 42
path_to_save_np = '/home/daniil/gans-mcmc/saved_numpy_arrays'
path_to_save = path_to_save_np
method_name = 'ula_dcgan_cifar_recalc'
every_step = 50
dataset = "cifar10"
calculate_is = True
calculate_msid_train = False
calculate_msid_test = True

ula_statistics = calculate_images_statistics(z_agg_step = load_np, 
                                             G = G,
                                             device = device, 
                                             batch_size = batch_size,
                                             path_to_save = path_to_save,
                                             path_to_save_np = path_to_save_np,
                                             method_name = method_name,
                                             random_seed = random_seed,
                                             every_step = every_step,
                                             dataset = dataset,
                                             calculate_is = calculate_is,
                                             calculate_msid_train = calculate_msid_train,
                                             calculate_msid_test = calculate_msid_test)

Files already downloaded and verified
Files already downloaded and verified
------------------------------------
step = 0
sample size = (50000, 100)
sample size after deleteting nans = (50000, 100)
start to calculate inception score...
Start to make predictions
Start to compute KL divergence
ula_dcgan_cifar_recalc mean inception score = 5.458847549031244, std inception score = 0.07988135484834574
time for inception calculation = 303.922s
start to calculate FID score for test cifar10...


100it [00:01, 63.53it/s]


shape of generated images = (10000, 3, 32, 32)
shape of real images = (10000, 3, 32, 32)


  1%|          | 2/200 [00:00<00:16, 11.80it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_pretrained_fake_test_step_0.npy


...

shape of generated images = (50000, 3, 32, 32)
shape of real images = (50000, 3, 32, 32)


  0%|          | 2/1000 [00:00<01:26, 11.59it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/ula_dcgan_cifar_recalc_pretrained_fake_train_step_1000.npy


fid: 100%|██████████| 10/10 [01:16<00:00,  7.68s/it, mean=41.9]

FID score for train cifar10 with ula_dcgan_cifar_recalc: mean 41.87512163122073, std 0.2449252218579044
time for FID calculation on train = 257.121s


In [8]:
ula_statistics

{'inception_scores_mean': array([5.45884755, 5.64533247, 5.93302675, 6.11075721, 6.26332333,
        6.33129223, 6.34728343, 6.44997478, 6.46983357, 6.46627593,
        6.49517636, 6.49804426, 6.50062137, 6.486893  , 6.45767812,
        6.48513536, 6.39664369, 6.416372  , 6.36567891, 6.35009992,
        6.3099507 ]),
 'inception_scores_std': array([0.07988135, 0.06556845, 0.0751358 , 0.09356253, 0.09139792,
        0.0363808 , 0.06738579, 0.06851357, 0.08193226, 0.12731706,
        0.08449642, 0.06955036, 0.05162749, 0.05048255, 0.08984082,
        0.06764603, 0.06941958, 0.0837918 , 0.07713408, 0.08823519,
        0.06783956]),
 'fid_scores_mean_train': array([35.71696744, 41.23658666, 34.94264839, 29.99145377, 26.83238766,
        25.4330189 , 24.85777145, 24.91446835, 25.51014069, 26.43138298,
        27.65998465, 28.87746921, 30.41964352, 31.89948227, 33.36489927,
        34.72285885, 36.21797335, 37.6261386 , 38.9148296 , 40.3451899 ,
        41.87512163]),
 'fid_scores_mean_test'

In [10]:
names_list = ["inception_scores_mean", "inception_scores_std",
              "fid_scores_mean_train", "fid_scores_mean_test",
              "fid_scores_std_train", "fid_scores_std_test",
              "msid_scores_mean_train", "msid_scores_mean_test",
              "msid_scores_std_train", "msid_scores_std_test"
             ]

In [11]:
method_name = 'ula_dcgan_cifar_recalc'
path_to_save_np = '/home/daniil/gans-mcmc/saved_numpy_arrays'
load_arrays = []
for i in range(len(names_list)):
    cur_score_path = os.path.join(path_to_save_np,
                                  f"{method_name}_{names_list[i]}.npy")
    load_arrays.append(np.load(cur_score_path))

## iSIR

In [6]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

batch_size = 200
n = 50000
n_steps = 500
N = 5

method_name = 'sir_dcgan_cifar_recalc'
path_to_save = '/home/daniil/gans-mcmc/saved_numpy_arrays'
file_name = f'{method_name}_N_{N}_nsteps_{n_steps}'
every_step = 50
continue_z = None

z_last_np, zs = sir_independent_sampling(target_gan, proposal, batch_size, n,
                                         path_to_save, file_name, every_step,
                                         continue_z, n_steps, N)

  0%|          | 1/250 [01:05<4:32:31, 65.67s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_N_5_nsteps_500_batch_num_0.npy was saved, file shape = (11, 200, 100)


  1%|          | 2/250 [02:14<4:35:12, 66.58s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_N_5_nsteps_500_batch_num_200.npy was saved, file shape = (11, 200, 100)


  1%|          | 3/250 [03:25<4:40:03, 68.03s/it]

...

file /home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_N_5_nsteps_500_batch_num_49400.npy was saved, file shape = (11, 200, 100)


100%|█████████▉| 249/250 [4:56:20<01:11, 71.49s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_N_5_nsteps_500_batch_num_49600.npy was saved, file shape = (11, 200, 100)


100%|██████████| 250/250 [4:57:31<00:00, 71.41s/it]

file /home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_N_5_nsteps_500_batch_num_49800.npy was saved, file shape = (11, 200, 100)


In [6]:
path_to_save = '/home/daniil/gans-mcmc/saved_numpy_arrays'
batch_size = 200
n = 50000
method_name = 'sir_dcgan_cifar_recalc'
n_steps = 500
N = 5
file_name = f'{method_name}_N_{N}_nsteps_{n_steps}'
load_np = load_data_from_batches(n, batch_size,
                                 path_to_save, file_name)

100%|██████████| 250/250 [00:00<00:00, 1422.20it/s]


In [8]:
load_np.shape

(11, 50000, 100)

In [9]:
batch_size = 100
random_seed = 42
path_to_save_np = '/home/daniil/gans-mcmc/saved_numpy_arrays'
path_to_save = path_to_save_np
method_name = 'sir_dcgan_cifar_recalc_stats'
every_step = 50
dataset = "cifar10"
calculate_is = True
calculate_msid_train = False
calculate_msid_test = True

ula_statistics = calculate_images_statistics(z_agg_step = load_np, 
                                             G = G,
                                             device = device, 
                                             batch_size = batch_size,
                                             path_to_save = path_to_save,
                                             path_to_save_np = path_to_save_np,
                                             method_name = method_name,
                                             random_seed = random_seed,
                                             every_step = every_step,
                                             dataset = dataset,
                                             calculate_is = calculate_is,
                                             calculate_msid_train = calculate_msid_train,
                                             calculate_msid_test = calculate_msid_test)

Files already downloaded and verified
Files already downloaded and verified
------------------------------------
step = 0
sample size = (50000, 100)
sample size after deleteting nans = (50000, 100)
start to calculate inception score...
Start to make predictions
Start to compute KL divergence
sir_dcgan_cifar_recalc_stats mean inception score = 5.477831259235616, std inception score = 0.05282534003607535
time for inception calculation = 306.318s
start to calculate FID score for test cifar10...


100it [00:01, 61.88it/s]


shape of generated images = (10000, 3, 32, 32)
shape of real images = (10000, 3, 32, 32)


  1%|          | 2/200 [00:00<00:16, 11.76it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_stats_pretrained_fake_test_step_0.npy


...

shape of generated images = (50000, 3, 32, 32)
shape of real images = (50000, 3, 32, 32)


  0%|          | 2/1000 [00:00<01:26, 11.52it/s]

/home/daniil/gans-mcmc/saved_numpy_arrays/sir_dcgan_cifar_recalc_stats_pretrained_fake_train_step_500.npy


fid: 100%|██████████| 10/10 [01:17<00:00,  7.79s/it, mean=28.5]

FID score for train cifar10 with sir_dcgan_cifar_recalc_stats: mean 28.479323786689594, std 0.09998309430502805
time for FID calculation on train = 258.925s
